# Imports

In [56]:
import sys

from sympy.testing.runtests import method

modules_to_reload = [
    "src.utils.method_loggers",
    "src.utils.method_runners",
    "src.utils.metrics_calculators",
    "src.utils.tensor_handlers",
    "src.utils.trackers",
    "src.utils.video_controller",
    "src.utils.optimal_rank_finders",
]

for module in modules_to_reload:
    if module in sys.modules:
        del sys.modules[module]

%load_ext memory_profiler
%load_ext autoreload
%autoreload 2

import gc
import os

import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import torch
from tqdm import tqdm
from torch.nn import Conv2d, ConvTranspose2d
from itertools import product
from src.utils.metrics_calculators import IMetricCalculator, CompressionRatioTensorLyTuckerCalculator
from functools import partial
from scipy.optimize import OptimizeResult
from scipy.optimize import differential_evolution
import time

np.random.seed(42)
os.environ["OPENBLAS_NUM_THREADS"] = "8"
os.environ["MKL_NUM_THREADS"] = "8"

import tensorly as tl
from dotenv import load_dotenv

load_dotenv()

tucker_args = {
    "svd": "truncated_svd",
    "init": "svd",
    "random_state": 42,
}

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# get tensor - layers of NN

In [57]:
gan = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub', 'DCGAN', pretrained=True, useGPU=False)


def extract_layers(model, tensors=None) -> list[torch.Tensor]:
    if tensors is None: tensors = []

    for child in model.children():
        if isinstance(child, (Conv2d, ConvTranspose2d)):
            layer_weights = child.weight
            size = layer_weights.shape
            layer_weights = layer_weights.reshape(size[0], size[1], size[2] * size[3])
            tensors.append(layer_weights)
        else:
            extract_layers(child, tensors=tensors)

    return tensors


tensors = extract_layers(gan.netG)
print(len(tensors))
for tensor in tensors: print(tensor.shape)

tensors = [tensors[3]]

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorch_GAN_zoo_hub


Average network found !
5
torch.Size([120, 512, 16])
torch.Size([512, 256, 16])
torch.Size([256, 128, 16])
torch.Size([128, 64, 16])
torch.Size([64, 3, 16])


# funcs

In [58]:
def calculate_tucker_bounds_for_layer_for_nn(shape: tuple | list) -> list:
    """
    Calculate the bounds for Tucker ranks of a tensor based on its shape.

    Parameters
    ----------
    shape : tuple[int, ...] | list[int]
        The shape of the tensor as a list or tuple of integers.
        Each element represents the size of the tensor along a corresponding dimension.

    Returns
    -------
    list[tuple[int, int]]
        A list of rank bounds for the Tucker decomposition.
        Each element is a tuple (r_min, r_max), where:
        - r_min is always 1 for all modes except the last.
        - r_max is the upper bound for the Tucker rank along the corresponding mode.

    Example
    -------
    >>> res = calculate_tucker_bounds_for_layer_for_nn((3, 4, 5))
    [(1, 3), (1, 4), (5, 5)]

    """
    return [(1, dim) for dim in shape[:-1]] + [(shape[-1], shape[-1])]

In [59]:
def compression_ratio_nn(tensor, ranks: list[int] | tuple[int, int]) -> float:
    """
    Returns the custom compression ratio of the layer of neural network after Tucker decomposition.

    Parameters
    ----------
    tensor : np.ndarray
        The original tensor.
    ranks : list[int] | tuple[int, int]
        The Tucker ranks for decomposition.

    Returns
    -------
    float
        The computed compression ratio.

    """
    size = tensor.shape
    size1 = size[0] * ranks[0]
    size2 = ranks[0] * ranks[1] * size[2] ** 2
    size3 = size[1] * ranks[1]
    return (size1 + size2 + size3) / (size[0] * size[1] * size[2] ** 2)

# Calculate search area

In [60]:
target_compression_ratio_for_graphs = 50.0
frobenius_error_coef_for_graphs = 1.0
compression_ratio_coef_for_graphs = 10.0

search_area_tensors_results = {}

total_iterations = 0
rank_ranges_per_layer = []

for tensor in tensors:
    tensor_tucker_bounds = calculate_tucker_bounds_for_layer_for_nn(tensor.shape)
    rank_ranges = [range(bound[0], bound[1] + 1) for bound in tensor_tucker_bounds]
    rank_ranges_per_layer.append(rank_ranges)
    total_iterations += np.prod([len(r) for r in rank_ranges])

with tqdm(total=total_iterations, desc="Processing All Layers") as pbar:
    for index, (tensor, rank_ranges) in enumerate(zip(tensors, rank_ranges_per_layer)):
        tqdm_iterable = product(*rank_ranges)

        search_area_tensors_results[index] = {}

        with tl.backend_context("pytorch"):
            tensor_cuda = tl.tensor(tensor).to("cuda")

            for rank_combination in tqdm_iterable:
                test_rank = list(rank_combination)
                internal_indices = test_rank[0:2]

                try:
                    method_result = tl.decomposition.tucker(tensor_cuda, rank=test_rank, **tucker_args)
                    reconstructed_tensor = tl.tucker_to_tensor(method_result)

                    target_compression_ratio = target_compression_ratio_for_graphs / 100

                    frobenius_error = (tl.norm(reconstructed_tensor - tensor_cuda) / tl.norm(tensor_cuda)).item()

                    custom_compression_ratio = compression_ratio_nn(tensor=reconstructed_tensor, ranks=test_rank)
                    custom_compression_penalty = (target_compression_ratio - custom_compression_ratio) ** 2

                    compression_ratio = CompressionRatioTensorLyTuckerCalculator.calculate(tensor_cuda,
                                                                                           method_result) / 100
                    compression_penalty = (target_compression_ratio - compression_ratio) ** 2

                    loss_function_result = (
                            frobenius_error_coef_for_graphs * frobenius_error
                            + compression_ratio_coef_for_graphs * compression_penalty
                    )

                    custom_loss_function_result = (
                            frobenius_error_coef_for_graphs * frobenius_error
                            + compression_ratio_coef_for_graphs * custom_compression_penalty
                    )

                    search_area_tensors_results[index][tuple(internal_indices)] = {
                        "rank": test_rank,
                        "frobenius_error": frobenius_error,
                        "compression_ratio": compression_ratio,
                        "compression_penalty": compression_penalty,
                        "loss_function_result": loss_function_result,
                        "custom_compression_ratio": custom_compression_ratio,
                        "custom_compression_penalty": custom_compression_penalty,
                        "custom_loss_function_result": custom_loss_function_result,
                    }

                except Exception as e:
                    print(e)

                finally:
                    torch.cuda.synchronize()
                    del method_result, reconstructed_tensor
                    torch.cuda.empty_cache()
                    gc.collect()

                # Обновляем общий tqdm после каждой итерации
                pbar.update(1)

        torch.cuda.synchronize()
        del tensor_cuda
        torch.cuda.empty_cache()
        gc.collect()

torch.cuda.synchronize()
torch.cuda.empty_cache()
gc.collect() 

Processing All Layers: 100%|██████████| 8192/8192 [34:47<00:00,  3.92it/s]


0

# Graph of search areas

In [61]:
metrics = ["frobenius_error", "compression_ratio", "custom_compression_ratio", "compression_penalty",
           "custom_compression_penalty", "loss_function_result", "custom_loss_function_result"]

for layer_index, search_area_tensor_results in search_area_tensors_results.items():

    tensor_name = f"conv_layer_{layer_index}"

    internal_indices = np.array(list(search_area_tensor_results.keys()))

    metric_dict = {tuple(idx): search_area_tensor_results[idx] for idx in search_area_tensor_results}

    figs = []

    for metric in metrics:
        z_values = np.array([search_area_tensor_results[key].get(metric, np.nan) for key in search_area_tensor_results])
        x_indices = internal_indices[:, 0]
        y_indices = internal_indices[:, 1]

        local_min_points = []

        for i, (x, y) in enumerate(zip(x_indices, y_indices)):
            z = z_values[i]

            neighbors = [
                (x - 1, y), (x + 1, y),
                (x, y - 1), (x, y + 1)
            ]

            is_local_min = all(
                (neighbor not in metric_dict or metric_dict[neighbor].get(metric, np.inf) >= z)
                for neighbor in neighbors
            )

            if is_local_min:
                local_min_points.append((x, y, z))

        x_min, y_min, z_min = zip(*local_min_points) if local_min_points else ([], [], [])

        fig = go.Figure()

        fig.add_trace(go.Scatter3d(
            x=x_indices,
            y=y_indices,
            z=z_values,
            mode="markers",
            marker={"size": 5, "color": z_values, "colorscale": "Viridis", "opacity": 0.8},
            name=metric
        ))

        fig.add_trace(go.Scatter3d(
            x=x_min,
            y=y_min,
            z=z_min,
            mode="markers+text",
            marker={"size": 6, "color": "red", "symbol": "diamond"},
            text=[f"min: {val:.6f}" for val in z_min],
            textposition="top center",
            name="Local Minima"
        ))

        fig.update_layout(
            title=f"Search area for example tensor {tensor_name} of {metric.replace('_', ' ').title()}",
            scene={
                "xaxis_title": "Rank Index 1",
                "yaxis_title": "Rank Index 2",
                "zaxis_title": metric.replace("_", " ").title(),
                "yaxis": {"tickmode": "array", "tickvals": list(set(y_indices.astype(int)))}
            },
            margin={"l": 0, "r": 0, "t": 40, "b": 0},
            template="plotly_white",
            showlegend=False,
        )

        figs.append(fig)

    html_str = ""
    for fig in figs:
        html_str += go.Figure(fig).to_html(full_html=False, include_plotlyjs=False)

    html_file = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    </head>
    <body>
    <h1>Search area by some metrics</h1>
    {html_str}
    </body>
    </html>
    """

    output_path = f"../.cache/data_analyze/optimization_algs_for_tucker_search_area_{tensor_name}.html"
    with open(output_path, "w", encoding="utf-8") as f:  # noqa: PTH123
        f.write(html_file)

# Calculate optimal rank and path of it

In [62]:
frobenius_error_coef_algs = 1.0
compression_ratio_coef_algs = 10.0

target_compression_ratio_algs = 50.0

scipy_algs_tensors_results = {}

In [63]:
def loss_function(
        rank: list,
        tensor: np.ndarray,
        target_compression_ratio: float,
        method_args: dict[str, str],
        frobenius_error_coef: float = 1.0,
        compression_ratio_coef: float = 10.0,
):
    try:
        with tl.backend_context("pytorch"):
            tensor_cuda = tl.tensor(tensor).to("cuda")

            method_result = tl.decomposition.tucker(tensor_cuda, rank=rank, **method_args)
            reconstructed_tensor = tl.tucker_to_tensor(method_result)

            target_compression_ratio /= 100
            frobenius_error = (tl.norm(reconstructed_tensor - tensor_cuda) / tl.norm(tensor_cuda)).item()
            compression_ratio = CompressionRatioTensorLyTuckerCalculator.calculate(tensor_cuda, method_result) / 100
            compression_penalty = (target_compression_ratio - compression_ratio) ** 2

            loss_function_result = (
                    frobenius_error_coef * frobenius_error
                    + compression_ratio_coef * compression_penalty
            )

        return loss_function_result

    except Exception as e:
        print(e)
        return float("inf")
    finally:
        torch.cuda.synchronize()
        del tensor, tensor_cuda, method_result, reconstructed_tensor, compression_ratio, compression_penalty
        torch.cuda.empty_cache()
        gc.collect()

In [64]:
def custom_loss_function(
        rank: list,
        tensor: np.ndarray,
        target_compression_ratio: float,
        method_args: dict[str, str],
        frobenius_error_coef: float = 1.0,
        compression_ratio_coef: float = 10.0,
):
    try:
        with tl.backend_context("pytorch"):
            tensor_cuda = tl.tensor(tensor).to("cuda")

            method_result = tl.decomposition.tucker(tensor_cuda, rank=rank, **method_args)
            reconstructed_tensor = tl.tucker_to_tensor(method_result)

            target_compression_ratio /= 100
            frobenius_error = (tl.norm(reconstructed_tensor - tensor_cuda) / tl.norm(tensor_cuda)).item()

            custom_compression_ratio = compression_ratio_nn(tensor=reconstructed_tensor, ranks=test_rank)

            custom_compression_penalty = (target_compression_ratio - custom_compression_ratio) ** 2

            custom_loss_function_result = (
                    frobenius_error_coef * frobenius_error
                    + compression_ratio_coef * custom_compression_penalty
            )

        return custom_loss_function_result

    except Exception as e:
        print(e)
        return float("inf")
    finally:
        torch.cuda.synchronize()
        del tensor, tensor_cuda, method_result, reconstructed_tensor, compression_ratio, compression_penalty
        torch.cuda.empty_cache()
        gc.collect()

In [65]:
def global_optimize_rank(
        tensor: np.ndarray,
        target_compression_ratio: float,
        method_args: dict[str, str],
        frobenius_error_coef: float = 1.0,
        compression_ratio_coef: float = 10.0,
        optimization_method: str = "differential_evolution",
        is_custom_loss_function: bool = False,
):
    def loss_wrapper(
            free_rank: list,
            tensor: np.ndarray,
            target_compression_ratio: float,
            frobenius_error_coef: float,
            compression_ratio_coef: float,
            method_args: dict[str, str],
    ) -> float:
        full_rank = list(np.clip(np.round(free_rank).astype(int), 1, None))
        try:
            loss = loss_function(
                rank=full_rank,
                tensor=tensor,
                target_compression_ratio=target_compression_ratio,
                method_args=method_args,
                frobenius_error_coef=frobenius_error_coef,
                compression_ratio_coef=compression_ratio_coef,
            )
        except Exception as e:
            print(e)
            loss = float("inf")
        return loss

    def custom_loss_wrapper(
            free_rank: list,
            tensor: np.ndarray,
            target_compression_ratio: float,
            frobenius_error_coef: float,
            compression_ratio_coef: float,
            method_args: dict[str, str],
    ) -> float:
        full_rank = list(np.clip(np.round(free_rank).astype(int), 1, None))
        try:
            loss = custom_loss_function(
                rank=full_rank,
                tensor=tensor,
                target_compression_ratio=target_compression_ratio,
                method_args=method_args,
                frobenius_error_coef=frobenius_error_coef,
                compression_ratio_coef=compression_ratio_coef,
            )
        except Exception as e:
            print(e)
            loss = float("inf")
        return loss
    
    def calculate_tucker_bounds_for_layer_for_nn(shape: tuple | list) -> list:
        """
        Calculate the bounds for Tucker ranks of a tensor based on its shape.
    
        Parameters
        ----------
        shape : tuple[int, ...] | list[int]
            The shape of the tensor as a list or tuple of integers.
            Each element represents the size of the tensor along a corresponding dimension.
    
        Returns
        -------
        list[tuple[int, int]]
            A list of rank bounds for the Tucker decomposition.
            Each element is a tuple (r_min, r_max), where:
            - r_min is always 1 for all modes except the last.
            - r_max is the upper bound for the Tucker rank along the corresponding mode.
    
        Example
        -------
        >>> res = calculate_tucker_bounds_for_layer_for_nn((3, 4, 5))
        [(1, 3), (1, 4), (5, 5)]
    
        """
        return [(1, dim) for dim in shape[:-1]] + [(shape[-1], shape[-1])]

    def calculate_metrics(
            tensor: np.ndarray,
            rank: list,
            method_args: dict[str, str],
            target_compression_ratio_percent: float = 50.0,
            frobenius_error_coef: float = 1.0,
            compression_ratio_coef: float = 10.0,
    ):
        with tl.backend_context("pytorch"):
            tensor_cuda = tl.tensor(tensor).to("cuda")

            method_result = tl.decomposition.tucker(tensor_cuda, rank=rank, **method_args)
            reconstructed_tensor = tl.tucker_to_tensor(method_result)

            target_compression_ratio = target_compression_ratio_percent / 100

            frobenius_error = (tl.norm(reconstructed_tensor - tensor_cuda) / tl.norm(tensor_cuda)).item()

            custom_compression_ratio = compression_ratio_nn(tensor=reconstructed_tensor, ranks=rank)
            custom_compression_penalty = (target_compression_ratio - custom_compression_ratio) ** 2

            compression_ratio = CompressionRatioTensorLyTuckerCalculator.calculate(tensor_cuda, method_result) / 100
            compression_penalty = (target_compression_ratio - compression_ratio) ** 2

            loss_function_result = (
                    frobenius_error_coef * frobenius_error
                    + compression_ratio_coef * compression_penalty
            )

            custom_loss_function_result = (
                    frobenius_error_coef * frobenius_error
                    + compression_ratio_coef * compression_penalty
            )

            metrics = {
                "rank": test_rank,
                "frobenius_error": frobenius_error,
                "compression_ratio": compression_ratio,
                "compression_penalty": compression_penalty,
                "loss_function_result": loss_function_result,
                "custom_compression_ratio": custom_compression_ratio,
                "custom_compression_penalty": custom_compression_penalty,
                "custom_loss_function_result": custom_loss_function_result,
            }

        torch.cuda.synchronize()
        del tensor_cuda, tensor, method_result, reconstructed_tensor
        torch.cuda.empty_cache()
        gc.collect()

        return metrics

    class OptimizationLogger:
        def __init__(
                self,
                tensor: np.ndarray,
                method_args: dict[str, str],
                target_compression_ratio: float = 50.0,
                frobenius_error_coef: float = 1.0,
                compression_ratio_coef: float = 10.0,
        ):
            self.logs = []
            self.current_iteration = -1

            self.tensor = tensor
            self.method_args = method_args
            self.target_compression_ratio = target_compression_ratio
            self.frobenius_error_coef = frobenius_error_coef
            self.compression_ratio_coef = compression_ratio_coef

        def calculate_metrics(
                self,
                rank: list,
        ) -> dict[str, float]:
            return calculate_metrics(
                tensor=self.tensor,
                rank=rank,
                method_args=self.method_args,
                target_compression_ratio_percent=self.target_compression_ratio,
                frobenius_error_coef=self.frobenius_error_coef,
                compression_ratio_coef=self.compression_ratio_coef,
            )

        def callback(self, intermediate_result: OptimizeResult):
            self.current_iteration += 1

            rank = list(np.round(intermediate_result.x).astype(int))
            metrics = self.calculate_metrics(rank=rank)

            self.logs.append(
                {
                    "step": self.current_iteration,
                    "rank": rank,
                    "metrics": metrics,
                    "raw_results": intermediate_result,
                }
            )
            print(f"\n=== Iteration {self.current_iteration} complete ===", f"New rank estimate: {rank}\n", sep="\n")

    optimization_logger = OptimizationLogger(
        tensor=tensor,
        method_args=method_args,
        target_compression_ratio=target_compression_ratio,
        frobenius_error_coef=frobenius_error_coef,
        compression_ratio_coef=compression_ratio_coef,
    )

    loss_function_fixed = partial(
        loss_wrapper,
        tensor=tensor,
        target_compression_ratio=target_compression_ratio,
        method_args=method_args,
        frobenius_error_coef=frobenius_error_coef,
        compression_ratio_coef=compression_ratio_coef,
    ) if is_custom_loss_function is False else partial(
        custom_loss_wrapper,
        tensor=tensor,
        target_compression_ratio=target_compression_ratio,
        method_args=method_args,
        frobenius_error_coef=frobenius_error_coef,
        compression_ratio_coef=compression_ratio_coef,
    )

    # params
    is_bounds_variable_usable = [
        "differential_evolution",
    ]

    is_callback_variable_not_usable = []

    free_bounds = calculate_tucker_bounds_for_layer_for_nn(
        tensor.shape) if optimization_method in is_bounds_variable_usable else None

    callback_param = (
        optimization_logger.callback if optimization_method not in is_callback_variable_not_usable else None
    )

    if optimization_method == "differential_evolution":
        optimization_kwargs_differential_evolution = {

            "func": loss_function_fixed,
            "bounds": free_bounds,

            "strategy": "best1bin",
            "maxiter": 50,
            "popsize": 10,
            "tol": 0.01,
            "atol": 0.001,
            "mutation": (0.3, 0.7),
            "recombination": 0.9,
            "init": "latinhypercube",
            "polish": True,

            "workers": 1,
            "updating": "immediate",  # {‘immediate’ - when 1 worker, ‘deferred’ - when more than 1 worker}

            "callback": callback_param,
            "disp": True,
        }

        result = differential_evolution(**optimization_kwargs_differential_evolution)

        optimal_rank = list(np.clip(np.round(result.x).astype(int), 1, None))
        final_loss = result.fun

    return optimal_rank, final_loss, result, optimization_logger.logs

In [70]:
method = "differential_evolution"

for index, tensor in enumerate(tensors):

    scipy_algs_tensors_results[index] = {}

    print(
        f"Testing tensor: {index}",
        f"Testing optimization method: {method}",
        f"Tensor shape: {tensor.shape}",
        sep="\n",
        end="\n\n",
    )
    try:
        # check optimizer method
        start_time = time.perf_counter()
        optimal_rank, final_loss, minimize_result_differential_evolution, iteration_logs_differential_evolution = global_optimize_rank(
            tensor=tensor,
            target_compression_ratio=target_compression_ratio_algs,
            method_args=tucker_args,
            optimization_method=method,
            frobenius_error_coef=frobenius_error_coef_algs,
            compression_ratio_coef=compression_ratio_coef_algs,
        )
        elapsed_time = time.perf_counter() - start_time

        scipy_algs_tensors_results[index][method] = {
            "final_results": minimize_result_differential_evolution,
            "steps_results": iteration_logs_differential_evolution,
        }
    except Exception as e:
        print(f"Error with tensor {index} with method {method}: {e}")

Testing tensor: 0
Testing optimization method: differential_evolution
Tensor shape: torch.Size([128, 64, 16])

differential_evolution step 1: f(x)= 0.45152145624160767

=== Iteration 0 complete ===
New rank estimate: [80, 42, 16]

differential_evolution step 2: f(x)= 0.44451943039894104

=== Iteration 1 complete ===
New rank estimate: [74, 46, 16]

differential_evolution step 3: f(x)= 0.4444291889667511

=== Iteration 2 complete ===
New rank estimate: [78, 45, 16]

differential_evolution step 4: f(x)= 0.4412423372268677

=== Iteration 3 complete ===
New rank estimate: [71, 50, 16]

differential_evolution step 5: f(x)= 0.44120851159095764

=== Iteration 4 complete ===
New rank estimate: [72, 49, 16]

Polishing solution with 'L-BFGS-B'


In [74]:
iteration_logs_differential_evolution[0]['metrics']

{'rank': [128, 64, 16],
 'frobenius_error': 0.4503675103187561,
 'compression_ratio': 0.5107421875,
 'compression_penalty': 0.00011539459228515625,
 'loss_function_result': 0.45152145624160767,
 'custom_compression_ratio': 0.41632080078125,
 'custom_compression_penalty': 0.007002208381891251,
 'custom_loss_function_result': 0.45152145624160767}

In [75]:
iteration_logs_differential_evolution[1]['metrics']

{'rank': [128, 64, 16],
 'frobenius_error': 0.4430293142795563,
 'compression_ratio': 0.51220703125,
 'compression_penalty': 0.00014901161193847656,
 'loss_function_result': 0.44451943039894104,
 'custom_compression_ratio': 0.42144775390625,
 'custom_compression_penalty': 0.006170455366373062,
 'custom_loss_function_result': 0.44451943039894104}

In [79]:
metric_names = ["frobenius_error", "compression_ratio", "custom_compression_ratio", "compression_penalty",
                "custom_compression_penalty", "loss_function_result", "custom_loss_function_result"]


for layer_index, scipy_tensor_algs_results in scipy_algs_tensors_results.items():

    tensor_name = f"conv_layer_{layer_index}"
    internal_indices = np.array(list(search_area_tensors_results[layer_index].keys()))

    for method_name, method_results in scipy_tensor_algs_results.items():
        frobenius_error_from_method, compression_ratio_from_method, custom_compression_ratio_from_method, compression_penalty_from_method, custom_compression_penalty_from_method, loss_from_method, custom_loss_from_method = [], [], [], [], [], [], []
        for method_steps_logs in method_results['steps_results']:
            frobenius_error_from_method.append(method_steps_logs["metrics"]["frobenius_error"])
            compression_ratio_from_method.append(method_steps_logs["metrics"]["compression_ratio"])
            compression_penalty_from_method.append(method_steps_logs["metrics"]["compression_penalty"])
            loss_from_method.append(method_steps_logs["metrics"]["loss_function_result"])
            custom_compression_ratio_from_method.append(method_steps_logs["metrics"]["custom_compression_ratio"])
            custom_compression_penalty_from_method.append(method_steps_logs["metrics"]["custom_compression_penalty"])
            custom_loss_from_method.append(method_steps_logs["metrics"]["custom_loss_function_result"])

        figs = []

        for metric, metric_data in zip(
                metric_names,
                [
                    frobenius_error_from_method,
                    compression_ratio_from_method,
                    custom_compression_ratio_from_method,
                    compression_penalty_from_method,
                    custom_compression_penalty_from_method,
                    loss_from_method,
                    custom_loss_from_method,
                ],
                strict=False,
        ):
            z_values = np.array(
                [search_area_tensors_results[layer_index][key].get(metric, np.nan) for key in
                 search_area_tensors_results[layer_index]])
            x_indices = internal_indices[:, 0]
            y_indices = internal_indices[:, 1]

            # Поиск локальных минимумов
            local_min_points = []

            metric_dict = {tuple(idx): search_area_tensors_results[layer_index][idx] for idx in
                           search_area_tensors_results[layer_index]}

            for i, (x, y) in enumerate(zip(x_indices, y_indices)):
                z = z_values[i]

                neighbors = [
                    (x - 1, y), (x + 1, y),  # По оси X
                    (x, y - 1), (x, y + 1)  # По оси Y
                ]

                is_local_min = all(
                    (neighbor not in metric_dict or metric_dict[neighbor].get(metric, np.inf) >= z)
                    for neighbor in neighbors
                )

                if is_local_min:
                    local_min_points.append((x, y, z))

            x_min, y_min, z_min = zip(*local_min_points) if local_min_points else ([], [], [])

            fig = go.Figure()

            # Основные точки
            fig.add_trace(go.Scatter3d(
                x=x_indices,
                y=y_indices,
                z=z_values,
                mode="markers",
                marker={"size": 5, "color": z_values, "colorscale": "Viridis", "opacity": 0.8},
            ))

            path_x = []
            path_y = []
            path_z = []

            for i, log in enumerate(method_results['steps_results']):
                rank = log["rank"]
                if metric == "frobenius_error":
                    z_value = frobenius_error_from_method[i]
                elif metric == "compression_ratio":
                    z_value = compression_ratio_from_method[i]
                elif metric == "compression_penalty":
                    z_value = compression_penalty_from_method[i]
                elif metric == "loss_function_result":
                    z_value = loss_from_method[i]
                elif metric == "custom_compression_ratio":
                    z_value = custom_compression_ratio_from_method[i]
                elif metric == "custom_compression_penalty":
                    z_value = custom_compression_penalty_from_method[i]
                elif metric == "custom_loss_function_result":
                    z_value = custom_loss_from_method[i]

                path_x.append(rank[1])
                path_y.append(rank[2])
                path_z.append(z_value)

                fig.add_trace(
                    go.Scatter3d(
                        x=[rank[1]],
                        y=[rank[2]],
                        z=[z_value],
                        mode="markers",
                        marker={
                            "size": 10 if i == 0 or i == len(method_results['steps_results']) - 1 else 5,
                            "color": "yellow" if i == 0 or i == len(method_results['steps_results']) - 1 else "red",
                            "opacity": 0.8,
                        },
                    )
                )

            # Добавляем локальные минимумы (выделенные точки)
            fig.add_trace(go.Scatter3d(
                x=x_min,
                y=y_min,
                z=z_min,
                mode="markers+text",
                marker={"size": 8, "color": "blue", "symbol": "diamond"},
                text=[f"min: {val:.6f}" for val in z_min],
                textposition="top center",
            ))

            fig.add_trace(
                go.Scatter3d(
                    x=path_x,
                    y=path_y,
                    z=path_z,
                    mode="lines+markers",
                    marker={"size": 5, "color": "red", "opacity": 0.8},
                    line={"color": "red", "width": 3},
                )
            )

            fig.update_layout(
                title=f"Search area for example tensor {tensor_name} of {metric.replace('_', ' ').title()} with {method_name} alg path",
                scene={
                    "xaxis_title": "Rank Index 1",
                    "yaxis_title": "Rank Index 2",
                    "zaxis_title": metric.replace("_", " ").title(),
                    "yaxis": {"tickmode": "array", "tickvals": list(set(y_indices.astype(int)))},
                },
                margin={"l": 0, "r": 0, "t": 40, "b": 0},
                template="plotly_white",
                showlegend=False,
            )

            figs.append(fig)

        html_str = ""
        for fig in figs:
            html_str += go.Figure(fig).to_html(full_html=False, include_plotlyjs=False)

        html_file = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        </head>
        <body>
        <h1>Search area by some metrics</h1>
        {html_str}
        </body>
        </html>
        """

        output_path = f"../.cache/data_analyze/optimization_algs_for_tensor_train_search_area_{tensor_name}_with_{method_name}_alg.html"
        with open(output_path, "w", encoding="utf-8") as f:  # noqa: PTH123
            f.write(html_file)